In [1]:
#testing connection
import http.client
import json

conn = http.client.HTTPSConnection("tools.htrc.illinois.edu")

headers = { 'Content-Type': "application/json" }

conn.request("GET", "/ef-api/volumes/mdp.49015000911785?pos=true&fields=htid%2Cfeatures.pages.body.tokenPosCount", headers=headers)

res = conn.getresponse()
data = res.read()

d = json.loads(data.decode("utf-8"))
next(iter(d['data']['features']['pages'][1]['body']['tokenPosCount'].items()))#already returns htid 

('University', {'NNP': 1})

In [2]:
#individual volumes
conn = http.client.HTTPSConnection("tools.htrc.illinois.edu")

headers = { 'Content-Type': "application/json" }

conn.request("HEAD", "/ef-api/volumes/loc.ark+=13960=t46q23w14", headers=headers)

res = conn.getresponse()

In [3]:
#individual volumes
conn = http.client.HTTPSConnection("tools.htrc.illinois.edu")

headers = { 'Content-Type': "application/json" }

conn.request("HEAD", "/ef-api/volumes/yul.12598968_000_00", headers=headers)

res = conn.getresponse()


In [4]:
conn = http.client.HTTPSConnection("tools.htrc.illinois.edu")

headers = { 'Content-Type': "application/json" }

conn.request("GET", "/ef-api/volumes/yul.12598968_000_00/metadata?fields=htid%2Cmetadata.pubDate", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"code":200,"data":{"htid":"yul.12598968_000_00","metadata":{"pubDate":1945}}}


In [5]:
import http.client
import json
conn = http.client.HTTPSConnection("tools.htrc.illinois.edu")

headers = { 'Content-Type': "application/json" }

conn.request("GET", "/ef-api/volumes/yul.12598968_000_00/metadata?fields=htid%2Cmetadata.pubDate", headers=headers)
#not specifying pages 
res = conn.getresponse()
data = res.read() #reads actual stream of data from transport protocol

data = json.loads(data.decode("utf-8")) #data.decode only returns string 
data['data']['htid']
data

{'code': 200,
 'data': {'htid': 'yul.12598968_000_00', 'metadata': {'pubDate': 1945}}}

In [6]:
#total tokencounts in a single volume
import http.client
import json

conn = http.client.HTTPSConnection("tools.htrc.illinois.edu")

headers = { 'Content-Type': "application/json" }

conn.request("GET", "/ef-api/volumes/loc.ark+=13960=t46q23w14?fields=features.pages.tokenCount", headers=headers)

res = conn.getresponse()
data = res.read()

data = json.loads(data.decode("utf-8"))
data
total = 0
for page in data['data']['features']['pages']:
    total += page['tokenCount']
print('total count in single volume:', total)

total count in single volume: 4055


In [7]:
#total token count for  multiple volumes (workset)
workset_id = '6424aa97330000a001a5dc9b'
import http.client

conn = http.client.HTTPSConnection("tools.htrc.illinois.edu")

headers = { 'Content-Type': "application/json" }

conn.request("GET", "/ef-api/worksets/6424aa97330000a001a5dc9b/volumes?fields=features.pages.tokenCount,htid", headers=headers)

res = conn.getresponse()
data = res.read()

data = json.loads(data.decode("utf-8"))
data
total = 0
htids = []
for volume in data['data']:
    print(volume['htid'])
    individual_vol = 0
    for page in volume['features']['pages']:
        total += page['tokenCount']
        individual_vol += page['tokenCount']
    htids.append(volume['htid'])
    print('total count for this volume:', individual_vol) #for individual volumes in the workset
        
print('total across all volumes:', total)


mdp.35112103187797
total count for this volume: 1381366
mdp.39015058744122
total count for this volume: 7422
uc1.$b684263
total count for this volume: 231986
uc1.32106011187561
total count for this volume: 7274
total across all volumes: 1628048


In [8]:
workset_id = '6424aa97330000a001a5dc9b'
import http.client
import json
from collections import defaultdict

conn = http.client.HTTPSConnection("tools.htrc.illinois.edu")

headers = { 'Content-Type': "application/json" }

conn.request("GET", "/ef-api/worksets/6424aa97330000a001a5dc9b/volumes?pos=false&fields=features.pages.tokenCount,features.pages.body.tokensCount,htid,metadata.title", headers=headers)

res = conn.getresponse()
data = res.read()

data = json.loads(data.decode("utf-8"))
data
d = defaultdict(lambda: 0) #dictionary
per_vol_dict = {}
titles = []
def updatedict(token_dict, results_dict):
    if token_dict:
        for token,count in token_dict.items():
            token = token.lower()
            results_dict[token] = results_dict[token]+count
            
for volume in data['data']:
    per_vol_dict[volume['htid']] = defaultdict(lambda: 0)
    for page in volume['features']['pages']:
        body = page.get('body')
        #header = page.get('header')
        #footer = page.get('footer')
        if body:
            updatedict(body['tokensCount'], d)
            updatedict(body['tokensCount'], per_vol_dict[volume['htid']])
    titles.append(volume['metadata']['title'])
titles

        # if header:
        #     updatedict(header['tokensCount'])
        # if footer:
        #     updatedict(footer['tokensCount'])
#print(d) #token counts for entire workset across header, body, footer
# make it more user friendly
# sort, print top 10
# use pandas 
#print(per_vol_dict)


['The Law times reports',
 'Highlights of ...',
 'The cruise of the Marchesa to Kamschatka & New Guinea.',
 'Bilder vom Erzählen : Gedichte /']

In [9]:
!pip install pandas
import pandas as pd


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [27]:
df = pd.DataFrame.from_dict(d, orient='index')
df = df.rename(columns={0:'count'})
print('tokens dataframe:', df)

tokens dataframe:                count
.&\-,@.            1
jv                 1
__..               1
,1»                1
_,..               1
...              ...
iioi               1
h/oo               1
ген                1
ekennzeichnet      1
lilbi              1

[48547 rows x 1 columns]


In [33]:
per_vol_dict.keys()
volume_df = {}
for htid in per_vol_dict:
    volume_df[htid] = pd.DataFrame.from_dict(per_vol_dict[htid], orient='index') #building df for an indi vol and putting it into dict
    volume_df[htid] = volume_df[htid].rename(columns={0:'count'})
    print(volume_df[htid].nlargest(10, 'count'))

       count
the   108434
,      65403
.      52244
of     50862
to     37182
and    30985
in     25902
that   23358
a      18025
is     14253
      count
,       382
the     328
.       307
and     230
of      176
in      144
to      121
a        87
data     85
for      74
     count
the  16346
,    15975
of    9493
.     8172
and   6337
to    4951
a     4869
in    4517
we    2562
is    2206
     count
und    286
.      243
der    237
die    216
in     147
ich    146
den    108
:      100
-       99
das     90


In [34]:
df.nlargest(10, 'count')

,count
the,125115
",",81798
.,60966
of,60538
to,42261
and,37553
in,30710
that,25009
a,22982
is,16496


In [13]:
!pip install nltk
import nltk


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [14]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aarthi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
#stopwords by language
stopwords = nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('german')
punctuations = [',','.',';','"','--',"``", ":", "-", "''", "-rrb-", "-lrb-"]
stopwords = stopwords+punctuations
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [35]:
# remove these stopwords from dataframe
for i in range(len(stopwords)):
    stopwords[i] = stopwords[i].replace('"', '')
stopwords
df.drop(stopwords, errors='ignore')
for i in volume_df:
    volume_df[i] = volume_df[i].drop(stopwords, errors = 'ignore')
    print(volume_df[i].nlargest(10, 'count'))
volume_df
#stopwords: words you want to ignore like 'the'

         count
act       5047
case      4694
v.        4263
company   3522
1         3458
upon      3250
l.        3194
's        2908
court     2851
rep.      2739
               count
data              85
satellite         53
2001              48
nesdis            42
noaa              39
environmental     33
new               32
's                30
goes              29
information       26
        count
one       786
us        695
little    578
two       496
island    462
's        402
time      390
would     335
almost    300
river     296
          count
...          68
(            54
)            53
nacht        32
schatten     21
zeit         19
meer         19
mehr         19
?            18
licht        17


{'mdp.35112103187797':          count
 .&\-,@.      1
 jv           1
 __..         1
 ,1»          1
 _,..         1
 ...        ...
 wrawn        1
 nvw          1
 31:4         1
 can-q        1
 qriy         1
 
 [36553 rows x 1 columns],
 'mdp.39015058744122':               count
 university        4
 *                16
 4                 7
 u                 2
 \                 6
 ...             ...
 866-732-2382      1
 phone             1
 toll-free         1
 ms                1
 hostetler         1
 
 [1913 rows x 1 columns],
 'uc1.$b684263':             count
 california      8
 university      3
 davis           4
 library         4
 marchesa       65
 ...           ...
 ucd             1
 ds507           1
 416980          1
 f.h.h.          1
 g9              1
 
 [15580 rows x 1 columns],
 'uc1.32106011187561':                count
 mufoñml            1
 wolfgang           6
 radierungen        2
 fischer            5
 hussel             6
 ...              ...
 h/oo 

In [36]:
df = df.drop(stopwords, errors='ignore')
df.nlargest(10, 'count')

,count
act,5062
case,4772
v.,4269
1,3624
company,3616
upon,3541
's,3340
one,3197
l.,3194
court,2857


In [37]:
new_list = []
for i in volume_df:
    new_string = ""
    for index, row in volume_df[i].iterrows():
        for i in range(0, row[0]):
            new_string += " " + index
    new_list.append(new_string)

In [19]:
!pip install scikit-learn



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [39]:
#intermediate step in the calculation of most unique words 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_counts = TfidfVectorizer()
a = tfidf_counts.fit_transform(new_list)
tfidf_counts.get_feature_names_out()
tfidf_df = pd.DataFrame(a.toarray(), index=titles, columns=tfidf_counts.get_feature_names_out())
tfidf_df

,00,000,0001,0007,001,0011,002,007,00i,00l,...,ﬂuhicature,ﬂuhimtnre,ﬂuhimtnrr,ﬂush,ﬂushing,ﬂwt,ﬂy,ﬂywheel,ﬂﬁgllég,ﬂﬂr
The Law times reports,0.004034,0.005946,0.005868,0.000073,0.00044,0.000073,0.000000,0.000073,0.000073,0.000073,...,0.000073,0.000073,0.000073,0.000073,0.00044,0.000073,0.001247,0.000073,0.000073,0.000073
Highlights of ...,0.000000,0.023084,0.000000,0.000000,0.00000,0.000000,0.006028,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
The cruise of the Marchesa to Kamschatka & New Guinea.,0.000000,0.010958,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
Bilder vom Erzählen : Gedichte /,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [38]:
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df

,level_0,level_1,0
0,159984,volume,Bilder vom Erzählen : Gedichte /
1,159984,token,nacht
2,159984,tfidf,0.314972
3,168072,volume,Bilder vom Erzählen : Gedichte /
4,168072,token,schatten
...,...,...,...
115,105383,token,found
116,105383,tfidf,0.114255
117,132315,volume,The cruise of the Marchesa to Kamschatka & New...
118,132315,token,would


In [22]:
# most commonly occuring uncommon word by volume titles
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'volume','level_1': 'token'})
tfidf_df = tfidf_df.sort_values(by=['volume','tfidf'], ascending=[True,False]).groupby(['volume']).head(10)
tfidf_df

,volume,token,tfidf
159984,Bilder vom Erzählen : Gedichte /,nacht,0.314972
168072,Bilder vom Erzählen : Gedichte /,schatten,0.206700
158831,Bilder vom Erzählen : Gedichte /,mehr,0.187015
177139,Bilder vom Erzählen : Gedichte /,zeit,0.187015
156959,Bilder vom Erzählen : Gedichte /,licht,0.167329
158813,Bilder vom Erzählen : Gedichte /,meer,0.147444
134235,Bilder vom Erzählen : Gedichte /,2ooo,0.108272
152401,Bilder vom Erzählen : Gedichte /,himmel,0.108272
168109,Bilder vom Erzählen : Gedichte /,schlaf,0.108272
175943,Bilder vom Erzählen : Gedichte /,weiß,0.108272
